# GDL for main graph rewiring and metrics comparison

## Import libraries

In [1]:
import sys
print(sys.executable)
from pathlib import Path
import torch
import torch_geometric
print(torch.__version__)
print(torch_geometric.__version__)

import matplotlib.pyplot as plt
from torch_geometric.datasets import TUDataset
# from GraphRicciCurvature.FormanRicci import FormanRicci

import networkx as nx
import numpy as np
from scipy.sparse.csgraph import laplacian
from scipy.linalg import pinv, eigvalsh

from utils.load_data import *
from evaluation.metrics import *
from evaluation.metrics_distance import *
from evaluation.curvature import *
from visualization.plots import *
from visualization.networkx_plot import *


/usr/local/bin/python3


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch_geometric/typing.py:128: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: dlopen(/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch_scatter/_version_cpu.so, 0x0006): Symbol not found: __ZN5torch3jit17parseSchemaOrNameERKNSt3__112basic_stringIcNS1_11char_traitsIcEENS1_9allocatorIcEEEEb
  Referenced from: <39DD586D-4BCB-3117-B50F-6A4C67CC40D3> /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch_scatter/_version_cpu.so
  Expected in:     <F8622D92-25A9-3A61-A089-C917FDA36C1B> /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch_geometric/typing.py:139: UserWarning: An issue occurr

2.2.2
2.7.0


# Rewiring our graphs

🚩 We need to respect the experimental details of hyperparameters describe on page 15

## Dynamic Graph Rewiring Method Code

In [ ]:
from ipywidgets import widgets, Tab, VBox, Output

dataset_names = ["MUTAG"]  # Default dataset selection
REWIRING_METHOD = "BORF"   # Default rewiring method
ENTIRE_GRAPH = False       # Default entire graph selection
metrics = "Normal"         

output = Output()  



data = widgets.Dropdown(
    options=["REDDIT-BINARY", "IMDB-BINARY", "MUTAG", "ENZYMES", "PROTEINS"],
    value="MUTAG",
    description="Dataset:"
)

rewired_method = widgets.Dropdown(
    options=["BORF", "SDRF", "FOSR", "DES", "PPR", "LASER", "UNREWIRED"],
    value="BORF",
    description="Rewiring Method:"
)

entire_graph = widgets.Checkbox(
    value=False,
    description="Entire Graph"
)


metric = widgets.Dropdown(
    options=["Distance", "Normal"],
    value="Normal",
    description="Metrics:"
)


def update_dataset(change):
    global dataset_names
    dataset_names = [change["new"]]
    with output:
        output.clear_output()
        print(f"Dataset Selected: {dataset_names}")

def update_rewiring_method(change):
    global REWIRING_METHOD
    REWIRING_METHOD = change["new"]
    with output:
        output.clear_output()
        print(f"Rewiring Method Selected: {REWIRING_METHOD}")

def update_entire_graph(change):
    global ENTIRE_GRAPH
    ENTIRE_GRAPH = change["new"]
    with output:
        output.clear_output()
        print(f"Entire Graph Selected: {ENTIRE_GRAPH}")

def update_metrics(change):
    global metrics
    metrics = change["new"]
    with output:
        output.clear_output()
        print(f"Metrics Selected: {metrics}")


data.observe(update_dataset, names="value")
rewired_method.observe(update_rewiring_method, names="value")
entire_graph.observe(update_entire_graph, names="value")
metric.observe(update_metrics, names="value")


tab_contents = [
    VBox([data]),
    VBox([rewired_method]),
    VBox([entire_graph]),
    VBox([metric])
]

tab = Tab(children=tab_contents)

# Set tab titles
for i, title in enumerate(["Dataset", "Rewiring Method", "Entire Graph", "Metrics"]):
    tab.set_title(i, title)

display(tab, output)


Output()

In [7]:

print(f"Selected dataset: {dataset_names}")
print(f"Selected rewiring method: {REWIRING_METHOD}")
print(f"Selected entire graph: {ENTIRE_GRAPH}")
print(f"Selected metrics: {metrics}")

Selected dataset: ['IMDB-BINARY']
Selected rewiring method: UNREWIRED
Selected entire graph: True
Selected metrics: Normal


In [ ]:
from rewiring.rewiring_call import *
from tqdm import tqdm
import tkinter as tk
from tkinter import ttk, messagebox

# dataset_names = ["REDDIT-BINARY", "IMDB-BINARY", "MUTAG", "ENZYMES", "PROTEINS"]
#                   0              1            2         3           4
# dataset_names = dataset_names[2]

# REWIRING_METHOD = ["BORF","SDRF", "FOSR", "DES","PPR", "LASER", "UNREWIRED"]
# #                   0       1       2       3      4      5         6
# REWIRING_METHOD = REWIRING_METHOD[0]

# # flag all graph rewiring or not
# ENTIRE_GRAPH = True

dataset_loader = GraphDatasetLoader(dataset_names)
loaded_datasets = dataset_loader.get_loaded_dataset_names()

all_metrics_df = []

for dataset_name in loaded_datasets:
    for rewiring_name in [REWIRING_METHOD]:
        print("Rewiring methods being used:", rewiring_name)
        print(f"\n🚀 Processing dataset: {dataset_name}")
        
        # for testing purposes we can test on one graph
        if not ENTIRE_GRAPH:  
            # Get first graph
            graphs = dataset_loader.first_graphs[dataset_name]
            # print("first graph", type(graphs))
        else:
            graphs = dataset_loader.datasets[dataset_name]  # Load full dataset

            
        for graph in tqdm(graphs):
            
            if isinstance(graphs, torch_geometric.data.data.Data):
                rewiring_method = rewiring_call(graphs, dataset_name)
            else:
                rewiring_method = rewiring_call(graph, dataset_name)
            
            if rewiring_name == "BORF":
                rewired_graph = rewiring_method.borf_rewiring()
            elif rewiring_name == "SDRF":
                rewired_graph = rewiring_method.sdrf_rewiring()
            elif rewiring_name == "FOSR":
                rewired_graph = rewiring_method.fosr_rewiring()
            elif rewiring_name == "LASER":
                rewired_graph = rewiring_method.laser_rewiring()
            elif rewiring_name == "DES":
                rewired_graph = rewiring_method.des_rewiring(dataset_loader)
            elif rewiring_name == "PPR":
                rewired_graph = rewiring_method.ppr_rewiring()
            elif rewiring_name == "UNREWIRED":
                
                G_nx = to_networkx(graph, to_undirected=True)  # Convert PyG graph to NetworkX
                print("G_nx", G_nx)
                rewired_graph = G_nx
           
            else:
                raise ValueError(f"Invalid rewiring method: {rewiring_name}")
            
            # Compute metrics for the rewired graph
            metrics_rewired = GraphMetrics(rewired_graph, dataset_name)
            df_metrics = metrics_rewired.get_all_metrics()
            df_metrics = pd.DataFrame([df_metrics]) 
            df_metrics["Rewiring Method"] = rewiring_name
            df_metrics["Dataset"] = dataset_name
            
            # Store and later save the metrics
            all_metrics_df.append(df_metrics)
    

# Convert results to DataFrame
final_df = pd.concat(all_metrics_df, ignore_index=True)

# Compute mean and standard deviation, excluding non-numeric columns
if ENTIRE_GRAPH:
    numeric_cols = final_df.select_dtypes(include=["number"])  
    avg_metrics = numeric_cols.mean().to_frame(name="Mean")  
    std_metrics = numeric_cols.std().to_frame(name="Std")  

    # Combine into a single DataFrame
    summary_df = pd.concat([avg_metrics, std_metrics], axis=1)

    # Format the output to display mean ± std in a single column
    summary_df["Formatted"] = summary_df.apply(lambda row: f"{row['Mean']:.6f} ± {row['Std']:.6f}", axis=1)

    # Save summary results correctly
    summary_output_csv = f"results/rewired_graph_avg_std_metrics_{dataset_name}_{REWIRING_METHOD}.csv"
    parent = Path(summary_output_csv).parent
    os.makedirs(parent, exist_ok=True)
    
    # If the path exists remove it
    if os.path.exists(summary_output_csv):
        os.remove(summary_output_csv)
    summary_df.to_csv(summary_output_csv, index=True)

    print(f"\n📂 Summary (Mean & Std) results saved to {summary_output_csv}.")
else:
    # Save individual rewiring results
    output_csv = f"results/rewired_graph_metrics_{dataset_name}.csv"
    # make sure the directory exists
    parent = Path(output_csv).parent
    os.makedirs(parent, exist_ok=True)
    
    final_df.to_csv(output_csv, index=False)
    print(f"\n📂 All rewiring results saved to {output_csv}.")

⚠️ Warning: 'IMDB-BINARY' not found. Did you mean 'IMDB-BINARY'?
✅ Dataset IMDB-BINARY already exists. Loading from disk...
✅ Converted 1000 graphs from IMDB-BINARY into NetworkX format.
Rewiring methods being used: UNREWIRED

🚀 Processing dataset: IMDB-BINARY


  0%|          | 0/1000 [00:00<?, ?it/s]

G_nx Graph with 20 nodes and 73 edges
Average degree of this graph is 7.3
Checking if graph is fully connected: True
Variance for degrees:  13.41
G_nx Graph with 32 nodes and 128 edges
Average degree of this graph is 8.0
Checking if graph is fully connected: True
Variance for degrees:  41.9375
G_nx Graph with 21 nodes and 81 edges
Average degree of this graph is 7.714285714285714
Checking if graph is fully connected: True
Variance for degrees:  10.489795918367346
G_nx Graph with 35 nodes and 133 edges
Average degree of this graph is 7.6
Checking if graph is fully connected: True
Variance for degrees:  38.239999999999995
G_nx Graph with 14 nodes and 56 edges
Average degree of this graph is 8.0
Checking if graph is fully connected: True
Variance for degrees:  5.0
G_nx Graph with 63 nodes and 211 edges
Average degree of this graph is 6.698412698412699
Checking if graph is fully connected: True
Variance for degrees:  50.496346686822875
G_nx Graph with 12 nodes and 66 edges
Average degree o

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/networkx/algorithms/assortativity/correlation.py:302: RuntimeWarning: invalid value encountered in scalar divide
  return float((xy * (M - ab)).sum() / np.sqrt(vara * varb))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/networkx/algorithms/assortativity/correlation.py:302: RuntimeWarning: invalid value encountered in scalar divide
  return float((xy * (M - ab)).sum() / np.sqrt(vara * varb))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/networkx/algorithms/assortativity/correlation.py:302: RuntimeWarning: invalid value encountered in scalar divide
  return float((xy * (M - ab)).sum() / np.sqrt(vara * varb))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/networkx/algorithms/assortativity/correlation.py:302: RuntimeWarning: invalid value encountered in scalar divide
  return float((xy * (M - ab)).sum() / np.sqr

G_nx Graph with 24 nodes and 105 edges
Average degree of this graph is 8.75
Checking if graph is fully connected: True
Variance for degrees:  11.604166666666666
G_nx Graph with 22 nodes and 59 edges
Average degree of this graph is 5.363636363636363
Checking if graph is fully connected: True
Variance for degrees:  12.41322314049587
G_nx Graph with 15 nodes and 50 edges
Average degree of this graph is 6.666666666666667
Checking if graph is fully connected: True
Variance for degrees:  5.155555555555556
G_nx Graph with 18 nodes and 153 edges
Average degree of this graph is 17.0
Checking if graph is fully connected: True
Variance for degrees:  0.0
NAN for Assortativity
G_nx Graph with 13 nodes and 44 edges
Average degree of this graph is 6.769230769230769
Checking if graph is fully connected: True
Variance for degrees:  8.331360946745564
G_nx Graph with 23 nodes and 141 edges
Average degree of this graph is 12.26086956521739
Checking if graph is fully connected: True
Variance for degrees:  

...

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/networkx/algorithms/assortativity/correlation.py:302: RuntimeWarning: invalid value encountered in scalar divide
  return float((xy * (M - ab)).sum() / np.sqrt(vara * varb))


NAN for Assortativity
G_nx Graph with 19 nodes and 75 edges
Average degree of this graph is 7.894736842105263
Checking if graph is fully connected: True
Variance for degrees:  12.199445983379501
G_nx Graph with 41 nodes and 476 edges
Average degree of this graph is 23.21951219512195
Checking if graph is fully connected: True
Variance for degrees:  121.43961927424152
G_nx Graph with 22 nodes and 67 edges
Average degree of this graph is 6.090909090909091
Checking if graph is fully connected: True
Variance for degrees:  11.173553719008265
G_nx Graph with 18 nodes and 59 edges
Average degree of this graph is 6.555555555555555
Checking if graph is fully connected: True
Variance for degrees:  7.4691358024691334
G_nx Graph with 13 nodes and 78 edges
Average degree of this graph is 12.0
Checking if graph is fully connected: True
Variance for degrees:  0.0
NAN for Assortativity
G_nx Graph with 21 nodes and 92 edges
Average degree of this graph is 8.761904761904763
Checking if graph is fully con

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/networkx/algorithms/assortativity/correlation.py:302: RuntimeWarning: invalid value encountered in scalar divide
  return float((xy * (M - ab)).sum() / np.sqrt(vara * varb))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/networkx/algorithms/assortativity/correlation.py:302: RuntimeWarning: invalid value encountered in scalar divide
  return float((xy * (M - ab)).sum() / np.sqrt(vara * varb))
 98%|█████████▊| 977/1000 [00:03<00:00, 332.29it/s]/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/networkx/algorithms/assortativity/correlation.py:302: RuntimeWarning: invalid value encountered in scalar divide
  return float((xy * (M - ab)).sum() / np.sqrt(vara * varb))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/networkx/algorithms/assortativity/correlation.py:302: RuntimeWarning: invalid value encountered in scalar d

G_nx Graph with 15 nodes and 41 edges
Average degree of this graph is 5.466666666666667
Checking if graph is fully connected: True
Variance for degrees:  6.115555555555554
G_nx Graph with 16 nodes and 39 edges
Average degree of this graph is 4.875
Checking if graph is fully connected: True
Variance for degrees:  8.859375
G_nx Graph with 33 nodes and 363 edges
Average degree of this graph is 22.0
Checking if graph is fully connected: True
Variance for degrees:  81.27272727272727
G_nx Graph with 12 nodes and 38 edges
Average degree of this graph is 6.333333333333333
Checking if graph is fully connected: True
Variance for degrees:  3.888888888888888
G_nx Graph with 14 nodes and 35 edges
Average degree of this graph is 5.0
Checking if graph is fully connected: True
Variance for degrees:  5.142857142857143
G_nx Graph with 13 nodes and 45 edges
Average degree of this graph is 6.923076923076923
Checking if graph is fully connected: True
Variance for degrees:  3.3017751479289945
G_nx Graph wit


/var/folders/vw/1sh30gx175312mgfglxh9j840000gn/T/ipykernel_504/654334966.py:86: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(all_metrics_df, ignore_index=True)
